# Bertoluzzo & Corrazza (2012) — Mercado multi‑agente RL simple
Simulación con M agentes. Cada agente tiene una señal paramétrica y actualiza un peso via gradiente sobre su Sharpe individual. El precio se mueve por shock exógeno + impacto agregado de órdenes (toy model).

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng(12)


In [ ]:

T = 4000; M = 5
# proceso base de retornos
eps = rng.normal(0, 0.01, size=T)

# parámetros de agentes
k = 6  # lags
W = rng.normal(0, 0.2, size=(M, k))
ema = 0.02; beta=1e-3; cost=0.0002; lr=0.02

def make_features(r, k):
    X = []
    for t in range(k, len(r)):
        X.append(r[t-k:t][::-1])
    return np.array(X)

r = np.zeros(T)
for t in range(1,T):
    r[t] = 0.05*r[t-1] + eps[t]

X = make_features(r, k)
S = np.zeros((M, len(X)))
SR_hist = []

# entrenamiento online multiagente (no competitivo explícito, pero coexisten)
m = np.zeros(M); v = np.ones(M)*1e-4
for t in range(len(X)):
    x = X[t]
    s = np.tanh(W @ x)                       # señales de M agentes
    ds = np.zeros(M) if t==0 else s - np.tanh(W @ X[t-1])
    pnl_i = s * r[t+k-1] - cost*np.abs(ds)   # pnl individual
    # momentos
    m = (1-ema)*m + ema*pnl_i
    v = (1-ema)*v + ema*(pnl_i - m)**2
    sr = m/np.sqrt(v + beta)
    SR_hist.append(sr.mean())
    # gradiente por agente
    ds_sign = np.tanh(1000*ds)
    dpnl_dwi = ((1 - s**2)[:,None] * (x[None,:]) * r[t+k-1][:None]  # primera parte
                 - cost*ds_sign[:,None]*(1 - s**2)[:,None]*(x[None,:]))
    # d SR / d pnl_i (vectorizado)
    d_sr_dm = 1/np.sqrt(v + beta)
    d_sr_dv = -0.5*m*(v+beta)**(-1.5)
    d_sr_dpnl = d_sr_dm*ema + d_sr_dv*(2*ema*(pnl_i - m))
    grad = (d_sr_dpnl[:,None]) * dpnl_dwi
    W += lr * grad

# Backtest posterior
S_final = np.tanh(W @ X.T)  # M x T'
pnl_all = S_final * r[k:][None,:]
eq = pnl_all.mean(axis=0).cumsum()

plt.figure(); plt.plot(eq); plt.title('Equity promedio de agentes'); plt.show()
